In [12]:
import numpy as np
import joblib

import params as p
import functions as f

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import RepeatedKFold

In [2]:
train, test, val = f.load_split_datasets(part='03')

In [3]:
X_train, y_train = f.split_data_X_y(train)
X_test, y_test = f.split_data_X_y(test)
X_val, y_val = f.split_data_X_y(val)

In [4]:
rf = joblib.load(p.OUTPUTS_PATH + 'model_rf.pkl')

In [5]:
f.evaluate_model(rf, X_test, y_test)

rmse : 209.2937000841431
mae : 131.43007100591717
r2 : 0.8928343612430919


In [6]:
rf_1 = RandomForestRegressor(random_state=p.RANDOM_STATE)

In [18]:
param_space = {
    "max_depth": np.arange(2, 10),
    # "n_estimators": np.arange(100, 5000, 50),
    # "min_samples_leaf": np.arange(2, 16, 1),
    # "min_samples_split": np.arange(0.1, 1, 0.05),
    # "max_features": ["auto", "sqrt", "log2"],
}

# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=p.RANDOM_STATE)

rs = RandomizedSearchCV(
    estimator = rf,
    param_distributions = param_space,
    n_jobs = -1,
    scoring='neg_mean_squared_error',
    cv = 3,
    random_state = p.RANDOM_STATE)

rs.fit(X_train, y_train)

/opt/anaconda3/envs/itai-dev/lib/python3.10/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9])},
                   random_state=73, scoring='neg_mean_squared_error')

In [19]:
tuned_params = rs.best_params_
tuned_params['random_state'] = p.RANDOM_STATE

In [20]:
rf_tuned = RandomForestRegressor(**tuned_params)
rf_tuned.fit(X_train, y_train)

RandomForestRegressor(max_depth=9, random_state=73)

In [21]:
f.evaluate_model(rf_tuned, X_test, y_test)

rmse : 235.22013351208037
mae : 150.33502223661534
r2 : 0.8646394128970033
